## Predicting Ice Cream Rating

Given *data about various ice creams and their ingredients*, let's try to predict the **average user rating** of a given ice cream.

We will use a linear regression model to make our predictions.

Data source: https://www.kaggle.com/datasets/tysonpo/ice-cream-dataset

### Importing Libraries

In [2]:
import numpy as np
import pandas as pd

import re
from nltk.stem import PorterStemmer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression, Ridge, Lasso

In [3]:
data = pd.read_csv('archive/combined/products.csv')
data

,brand,key,name,subhead,description,rating,rating_count,ingredients
0,bj,0_bj,Salted Caramel Core,Sweet Cream Ice Cream with Blonde Brownies & a...,Find your way to the ultimate ice cream experi...,3.7,208,"CREAM, SKIM MILK, LIQUID SUGAR (SUGAR, WATER),..."
1,bj,1_bj,Netflix & Chilll'd™,Peanut Butter Ice Cream with Sweet & Salty Pre...,There’s something for everyone to watch on Net...,4.0,127,"CREAM, SKIM MILK, LIQUID SUGAR (SUGAR, WATER),..."
2,bj,2_bj,Chip Happens,A Cold Mess of Chocolate Ice Cream with Fudge ...,Sometimes “chip” happens and everything’s a me...,4.7,130,"CREAM, LIQUID SUGAR (SUGAR, WATER), SKIM MILK,..."
3,bj,3_bj,Cannoli,Mascarpone Ice Cream with Fudge-Covered Pastry...,As a Limited Batch that captured the rapture o...,3.6,70,"CREAM, SKIM MILK, LIQUID SUGAR (SUGAR, WATER),..."
4,bj,4_bj,Gimme S’more!™,Toasted Marshmallow Ice Cream with Chocolate C...,It’s a gimme: there’s always room for s’more. ...,4.5,281,"CREAM, SKIM MILK, WATER, LIQUID SUGAR (SUGAR, ..."
...,...,...,...,...,...,...,...,...
236,breyers,64_breyers,CINNABON®,NaN,Calling all cinnamon roll lovers! We teamed up...,4.0,28,"MILK, CORN SYRUP, SUGAR, BROWN SUGAR, SOYBEAN ..."
237,breyers,65_breyers,CarbSmart™ Caramel Swirl Bar,NaN,Watching your grams of sugar or carbs? Try our...,4.7,18,"MILK, WATER, CARAMEL SWIRL, SUGAR, WATER, CORN..."
238,breyers,66_breyers,Layered Dessert S'mores,NaN,Calling all S'mores lovers! Make sure you don'...,2.5,31,"MILK, CORN SYRUP, SUGAR, WHEAT FLOUR, BUTTER, ..."
239,breyers,67_breyers,Layered Dessert Peach Cobbler,NaN,Love peach cobbler topped with Breyers®? Then ...,3.2,38,"MILK, CORN SYRUP, ENRICHED WHEAT FLOUR, WHEAT ..."


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241 entries, 0 to 240
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   brand         241 non-null    object 
 1   key           241 non-null    object 
 2   name          241 non-null    object 
 3   subhead       57 non-null     object 
 4   description   237 non-null    object 
 5   rating        241 non-null    float64
 6   rating_count  241 non-null    int64  
 7   ingredients   241 non-null    object 
dtypes: float64(1), int64(1), object(6)
memory usage: 15.2+ KB


### Preprocessing

In [5]:
df = data.drop(['key', 'name', 'subhead', 'description'], axis=1)
df

,brand,rating,rating_count,ingredients
0,bj,3.7,208,"CREAM, SKIM MILK, LIQUID SUGAR (SUGAR, WATER),..."
1,bj,4.0,127,"CREAM, SKIM MILK, LIQUID SUGAR (SUGAR, WATER),..."
2,bj,4.7,130,"CREAM, LIQUID SUGAR (SUGAR, WATER), SKIM MILK,..."
3,bj,3.6,70,"CREAM, SKIM MILK, LIQUID SUGAR (SUGAR, WATER),..."
4,bj,4.5,281,"CREAM, SKIM MILK, WATER, LIQUID SUGAR (SUGAR, ..."
...,...,...,...,...
236,breyers,4.0,28,"MILK, CORN SYRUP, SUGAR, BROWN SUGAR, SOYBEAN ..."
237,breyers,4.7,18,"MILK, WATER, CARAMEL SWIRL, SUGAR, WATER, CORN..."
238,breyers,2.5,31,"MILK, CORN SYRUP, SUGAR, WHEAT FLOUR, BUTTER, ..."
239,breyers,3.2,38,"MILK, CORN SYRUP, ENRICHED WHEAT FLOUR, WHEAT ..."


In [8]:
df = df.drop(df.query('rating_count <10').index, axis=0).reset_index(drop=True)
df

,brand,rating,rating_count,ingredients
0,bj,3.7,208,"CREAM, SKIM MILK, LIQUID SUGAR (SUGAR, WATER),..."
1,bj,4.0,127,"CREAM, SKIM MILK, LIQUID SUGAR (SUGAR, WATER),..."
2,bj,4.7,130,"CREAM, LIQUID SUGAR (SUGAR, WATER), SKIM MILK,..."
3,bj,3.6,70,"CREAM, SKIM MILK, LIQUID SUGAR (SUGAR, WATER),..."
4,bj,4.5,281,"CREAM, SKIM MILK, WATER, LIQUID SUGAR (SUGAR, ..."
...,...,...,...,...
231,breyers,4.0,28,"MILK, CORN SYRUP, SUGAR, BROWN SUGAR, SOYBEAN ..."
232,breyers,4.7,18,"MILK, WATER, CARAMEL SWIRL, SUGAR, WATER, CORN..."
233,breyers,2.5,31,"MILK, CORN SYRUP, SUGAR, WHEAT FLOUR, BUTTER, ..."
234,breyers,3.2,38,"MILK, CORN SYRUP, ENRICHED WHEAT FLOUR, WHEAT ..."


In [9]:
df = df.drop('rating_count', axis=1)
df

,brand,rating,ingredients
0,bj,3.7,"CREAM, SKIM MILK, LIQUID SUGAR (SUGAR, WATER),..."
1,bj,4.0,"CREAM, SKIM MILK, LIQUID SUGAR (SUGAR, WATER),..."
2,bj,4.7,"CREAM, LIQUID SUGAR (SUGAR, WATER), SKIM MILK,..."
3,bj,3.6,"CREAM, SKIM MILK, LIQUID SUGAR (SUGAR, WATER),..."
4,bj,4.5,"CREAM, SKIM MILK, WATER, LIQUID SUGAR (SUGAR, ..."
...,...,...,...
231,breyers,4.0,"MILK, CORN SYRUP, SUGAR, BROWN SUGAR, SOYBEAN ..."
232,breyers,4.7,"MILK, WATER, CARAMEL SWIRL, SUGAR, WATER, CORN..."
233,breyers,2.5,"MILK, CORN SYRUP, SUGAR, WHEAT FLOUR, BUTTER, ..."
234,breyers,3.2,"MILK, CORN SYRUP, ENRICHED WHEAT FLOUR, WHEAT ..."


In [12]:
df.loc[0, 'ingredients']

'CREAM, SKIM MILK, LIQUID SUGAR (SUGAR, WATER), WATER, BROWN SUGAR, SUGAR, MILK, WHEAT FLOUR, EGG YOLKS, CORN SYRUP, EGGS, BUTTER (CREAM, SALT), BUTTEROIL, PECTIN, SEA SALT, SOYBEAN OIL, VANILLA EXTRACT, GUAR GUM, SOY LECITHIN, BAKING POWDER (SODIUM ACID PYROPHOSPHATE, SODIUM BICARBONATE, CORN STARCH, MONOCALCIUM PHOSPHATE), BAKING SODA, SALT, CARRAGEENAN, LACTASE'

In [13]:
def process_ingredients(ingredients):
    ps = PorterStemmer()
    new_ingredients = re.sub(r'\(.*?\)', '', ingredients)
    new_ingredients = re.sub(r'CONTAINS:.*$', '', new_ingredients)
    new_ingredients = re.sub(r'\..*?:', ',', new_ingredients)
    new_ingredients = re.sub(r'( AND/OR )', ',', new_ingredients)
    new_ingredients = re.sub(r'( AND )', ',', new_ingredients)
    new_ingredients = new_ingredients.split(',')
    for i in range(len(new_ingredients)):
        new_ingredients[i] = new_ingredients[i].replace('†', '').replace('*', ' ').replace(')', '').replace('/', ' ')
        new_ingredients[i] = re.sub(r'^.+:', '', new_ingredients[i])
        new_ingredients[i] = ps.stem(new_ingredients[i].strip())
        if new_ingredients[i] == 'milk fat':
            new_ingredients[i] = 'milkfat'
    return new_ingredients

In [14]:
# Add all unique ingredients to all_ingredients
all_ingredients = set()

for row in df.iterrows():
    ingredients = process_ingredients(df.loc[row[0], 'ingredients'])
    for ingredient in ingredients:
        if ingredient not in all_ingredients:
            all_ingredients.add(ingredient)
all_ingredients.remove('')

In [17]:
all_ingredients

{'acesulfame potassium',
 'almond',
 'almond extract',
 'almond milk',
 'almonds roasted in vegetable oil',
 'anhydrous milkfat',
 'annatto',
 'apple juic',
 'artificial color',
 'artificial flavor',
 'ascorbic acid',
 'baking powd',
 'baking soda',
 'baking soda. contains milk',
 'balsamic vinegar',
 'banana',
 'banana pure',
 'barley malt',
 'beet juic',
 'belgian chocol',
 'black carrot concentr',
 'black cherri',
 'black raspberri',
 'black raspberry pure',
 'blackberry juice concentr',
 'bleached wheat flour',
 'blue 1',
 'blue 1 lak',
 'blue 2',
 'blue 2 lak',
 'blueberri',
 'blueberry puree concentr',
 'bourbon',
 'brown sugar',
 'butter',
 'butter oil',
 'butteroil',
 'calcium carbon',
 'calcium phosph',
 'cane sugar',
 'canola oil',
 'caramel',
 'caramel color',
 'caramel flavor',
 'caramel swirl',
 'caramelized sugar',
 'caramelized sugar syrup',
 'carnauba wax',
 'carob bean',
 'carob bean gum',
 'carob gum',
 'carrageenan',
 'carrot juice concentr',
 'carrot powd',
 'cheese

In [18]:
df

,brand,rating,ingredients
0,bj,3.7,"CREAM, SKIM MILK, LIQUID SUGAR (SUGAR, WATER),..."
1,bj,4.0,"CREAM, SKIM MILK, LIQUID SUGAR (SUGAR, WATER),..."
2,bj,4.7,"CREAM, LIQUID SUGAR (SUGAR, WATER), SKIM MILK,..."
3,bj,3.6,"CREAM, SKIM MILK, LIQUID SUGAR (SUGAR, WATER),..."
4,bj,4.5,"CREAM, SKIM MILK, WATER, LIQUID SUGAR (SUGAR, ..."
...,...,...,...
231,breyers,4.0,"MILK, CORN SYRUP, SUGAR, BROWN SUGAR, SOYBEAN ..."
232,breyers,4.7,"MILK, WATER, CARAMEL SWIRL, SUGAR, WATER, CORN..."
233,breyers,2.5,"MILK, CORN SYRUP, SUGAR, WHEAT FLOUR, BUTTER, ..."
234,breyers,3.2,"MILK, CORN SYRUP, ENRICHED WHEAT FLOUR, WHEAT ..."


In [19]:
process_ingredients(df.loc[0, 'ingredients'])

['cream',
 'skim milk',
 'liquid sugar',
 'water',
 'brown sugar',
 'sugar',
 'milk',
 'wheat flour',
 'egg yolk',
 'corn syrup',
 'egg',
 'butter',
 'butteroil',
 'pectin',
 'sea salt',
 'soybean oil',
 'vanilla extract',
 'guar gum',
 'soy lecithin',
 'baking powd',
 'baking soda',
 'salt',
 'carrageenan',
 'lactas']

In [56]:
y = df.loc[:, 'rating']
X = df.drop('rating', axis=1)

In [57]:
X

,brand,ingredients
0,bj,"CREAM, SKIM MILK, LIQUID SUGAR (SUGAR, WATER),..."
1,bj,"CREAM, SKIM MILK, LIQUID SUGAR (SUGAR, WATER),..."
2,bj,"CREAM, LIQUID SUGAR (SUGAR, WATER), SKIM MILK,..."
3,bj,"CREAM, SKIM MILK, LIQUID SUGAR (SUGAR, WATER),..."
4,bj,"CREAM, SKIM MILK, WATER, LIQUID SUGAR (SUGAR, ..."
...,...,...
231,breyers,"MILK, CORN SYRUP, SUGAR, BROWN SUGAR, SOYBEAN ..."
232,breyers,"MILK, WATER, CARAMEL SWIRL, SUGAR, WATER, CORN..."
233,breyers,"MILK, CORN SYRUP, SUGAR, WHEAT FLOUR, BUTTER, ..."
234,breyers,"MILK, CORN SYRUP, ENRICHED WHEAT FLOUR, WHEAT ..."


In [58]:
y

0      3.7
1      4.0
2      4.7
3      3.6
4      4.5
      ... 
231    4.0
232    4.7
233    2.5
234    3.2
235    2.8
Name: rating, Length: 236, dtype: float64

In [59]:
def onehot_encode(df, column, prefix):
    df = df.copy()
    dummies = pd.get_dummies(df[column], prefix=prefix, dtype=int)
    df = pd.concat([df, dummies], axis=1)
    df = df.drop(column, axis=1)
    return df

In [60]:
X = onehot_encode(X, 'brand', 'b')
X

,ingredients,b_bj,b_breyers,b_hd,b_talenti
0,"CREAM, SKIM MILK, LIQUID SUGAR (SUGAR, WATER),...",1,0,0,0
1,"CREAM, SKIM MILK, LIQUID SUGAR (SUGAR, WATER),...",1,0,0,0
2,"CREAM, LIQUID SUGAR (SUGAR, WATER), SKIM MILK,...",1,0,0,0
3,"CREAM, SKIM MILK, LIQUID SUGAR (SUGAR, WATER),...",1,0,0,0
4,"CREAM, SKIM MILK, WATER, LIQUID SUGAR (SUGAR, ...",1,0,0,0
...,...,...,...,...,...
231,"MILK, CORN SYRUP, SUGAR, BROWN SUGAR, SOYBEAN ...",0,1,0,0
232,"MILK, WATER, CARAMEL SWIRL, SUGAR, WATER, CORN...",0,1,0,0
233,"MILK, CORN SYRUP, SUGAR, WHEAT FLOUR, BUTTER, ...",0,1,0,0
234,"MILK, CORN SYRUP, ENRICHED WHEAT FLOUR, WHEAT ...",0,1,0,0


In [61]:
X['ingredients'] = X['ingredients'].apply(process_ingredients)

In [62]:
X

,ingredients,b_bj,b_breyers,b_hd,b_talenti
0,"[cream, skim milk, liquid sugar, water, brown ...",1,0,0,0
1,"[cream, skim milk, liquid sugar, water, sugar,...",1,0,0,0
2,"[cream, liquid sugar, skim milk, water, sugar,...",1,0,0,0
3,"[cream, skim milk, liquid sugar, water, corn s...",1,0,0,0
4,"[cream, skim milk, water, liquid sugar, sugar,...",1,0,0,0
...,...,...,...,...,...
231,"[milk, corn syrup, sugar, brown sugar, soybean...",0,1,0,0
232,"[milk, water, caramel swirl, sugar, water, cor...",0,1,0,0
233,"[milk, corn syrup, sugar, wheat flour, butter,...",0,1,0,0
234,"[milk, corn syrup, enriched wheat flour, wheat...",0,1,0,0


In [63]:
ingredient_columns = []

for ingredient_list in X['ingredients']:
    for ingredient in ingredient_list:
        if ingredient not in ingredient_columns:
            ingredient_columns.append(ingredient)

In [64]:
ingredient_columns

['cream',
 'skim milk',
 'liquid sugar',
 'water',
 'brown sugar',
 'sugar',
 'milk',
 'wheat flour',
 'egg yolk',
 'corn syrup',
 'egg',
 'butter',
 'butteroil',
 'pectin',
 'sea salt',
 'soybean oil',
 'vanilla extract',
 'guar gum',
 'soy lecithin',
 'baking powd',
 'baking soda',
 'salt',
 'carrageenan',
 'lactas',
 'peanut',
 'canola oil',
 'corn starch',
 'peanut oil',
 'cocoa powd',
 'invert cane sugar',
 'milkfat',
 'egg whit',
 'tapioca starch',
 'barley malt',
 'malted barley flour',
 'cocoa',
 'potato',
 'coconut oil',
 'corn syrup solid',
 'rice starch',
 'sunflower oil',
 'yeast extract',
 'natural flavor',
 'enzym',
 'baking soda. contains milk',
 'wheat',
 'soy',
 'dried cane syrup',
 'butter oil',
 'locust bean gum',
 'citric acid',
 'vanilla bean se',
 'lactic acid',
 'graham flour',
 'molass',
 'honey',
 'caramelized sugar syrup',
 'chocolate liquor',
 'tapioca flour',
 'peanut flour',
 'peanut extract',
 'cocoa butt',
 'roasted almond',
 'blackberry juice concentr',


In [65]:
ingredients_df = X['ingredients']
ingredients_df

0      [cream, skim milk, liquid sugar, water, brown ...
1      [cream, skim milk, liquid sugar, water, sugar,...
2      [cream, liquid sugar, skim milk, water, sugar,...
3      [cream, skim milk, liquid sugar, water, corn s...
4      [cream, skim milk, water, liquid sugar, sugar,...
                             ...                        
231    [milk, corn syrup, sugar, brown sugar, soybean...
232    [milk, water, caramel swirl, sugar, water, cor...
233    [milk, corn syrup, sugar, wheat flour, butter,...
234    [milk, corn syrup, enriched wheat flour, wheat...
235    [milk, corn syrup, sugar, enriched wheat flour...
Name: ingredients, Length: 236, dtype: object

In [66]:
mlb = MultiLabelBinarizer()

ingredients_df = pd.DataFrame(mlb.fit_transform(ingredients_df))

In [67]:
ingredients_df

,0,1,2,3,4,5,6,7,8,9,...,344,345,346,347,348,349,350,351,352,353
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
232,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
233,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
234,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [68]:
X

,ingredients,b_bj,b_breyers,b_hd,b_talenti
0,"[cream, skim milk, liquid sugar, water, brown ...",1,0,0,0
1,"[cream, skim milk, liquid sugar, water, sugar,...",1,0,0,0
2,"[cream, liquid sugar, skim milk, water, sugar,...",1,0,0,0
3,"[cream, skim milk, liquid sugar, water, corn s...",1,0,0,0
4,"[cream, skim milk, water, liquid sugar, sugar,...",1,0,0,0
...,...,...,...,...,...
231,"[milk, corn syrup, sugar, brown sugar, soybean...",0,1,0,0
232,"[milk, water, caramel swirl, sugar, water, cor...",0,1,0,0
233,"[milk, corn syrup, sugar, wheat flour, butter,...",0,1,0,0
234,"[milk, corn syrup, enriched wheat flour, wheat...",0,1,0,0


In [69]:
X = pd.concat([X, ingredients_df], axis=1)
X = X.drop('ingredients', axis=1)

In [70]:
X

,b_bj,b_breyers,b_hd,b_talenti,0,1,2,3,4,5,...,344,345,346,347,348,349,350,351,352,353
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
232,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
233,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
234,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [82]:
X.columns = X.columns.astype('str')

In [83]:
y

0      3.7
1      4.0
2      4.7
3      3.6
4      4.5
      ... 
231    4.0
232    4.7
233    2.5
234    3.2
235    2.8
Name: rating, Length: 236, dtype: float64

### Training

In [84]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=100)

In [85]:
X_train

,b_bj,b_breyers,b_hd,b_talenti,0,1,2,3,4,5,...,344,345,346,347,348,349,350,351,352,353
149,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
201,0,1,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
191,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
103,0,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
67,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [86]:
y_train

149    4.6
201    4.3
191    2.8
7      4.3
28     4.7
      ... 
87     4.3
103    3.5
67     4.5
24     3.8
8      4.7
Name: rating, Length: 165, dtype: float64

### Modelling (without Regularization)

In [87]:
model = LinearRegression()

model.fit(X_train, y_train)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [88]:
model.score(X_test, y_test)

-20.25673410075104

### Modelling with L2 (Ridge) Regression

In [104]:
l2_model = Ridge(alpha=1000.0)

l2_model.fit(X_train, y_train)

,alpha,1000.0
,fit_intercept,True
,copy_X,True
,max_iter,None
,tol,0.0001
,solver,'auto'
,positive,False
,random_state,None


In [105]:
l2_model.score(X_test, y_test)

-0.0636355289562438

### Modelling with L1 (Lasso) Regression

In [112]:
l1_model = Lasso(alpha=0.1)

l1_model.fit(X_train, y_train)

,alpha,0.1
,fit_intercept,True
,precompute,False
,copy_X,True
,max_iter,1000
,tol,0.0001
,warm_start,False
,positive,False
,random_state,None
,selection,'cyclic'


In [113]:
l1_model.score(X_test, y_test)

-0.06882219995590577